In [50]:
from PIL import Image
import glob

def resize_images(img_path):
	images=glob.glob(img_path + "/*.jpg")  
    
	print(len(images), " images to be resized.")

    # 파일마다 모두 128*128 사이즈로 바꾸어 저장합니다.
	target_size=(224,224)
	for img in images:
		old_img=Image.open(img)
		new_img=old_img.resize(target_size,Image.ANTIALIAS)
		new_img.save(img, "JPEG")
    
	print(len(images), " images resized.")

In [51]:
# 가위 이미지가 저장된 디렉토리 아래의 모든 jpg 파일을 읽어들여서
image_dir_path = r"moving\rockgame\dataset\scissors"
resize_images(image_dir_path)

print("가위 이미지 resize 완료!")

0  images to be resized.
0  images resized.
가위 이미지 resize 완료!


In [52]:
# 바위 이미지가 저장된 디렉토리 아래의 모든 jpg 파일을 읽어들여서
image_dir_path = r"moving\rockgame\dataset\rock"
resize_images(image_dir_path)

print("바위 이미지 resize 완료!")

0  images to be resized.
0  images resized.
바위 이미지 resize 완료!


In [53]:
# 보 이미지가 저장된 디렉토리 아래의 모든 jpg 파일을 읽어들여서
image_dir_path = r"moving\rockgame\dataset\paper"
resize_images(image_dir_path)

print("보 이미지 resize 완료!")

0  images to be resized.
0  images resized.
보 이미지 resize 완료!


In [54]:
# !pip install matplotlib
# !pip install scikit-learn
# !pip install scipy

In [55]:
import numpy as np
import os
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import train_test_split

def load_data(img_path, number_of_data=7491):  # 가위바위보 이미지 개수 총합에 주의하세요.
    # 가위 : 0, 바위 : 1, 보 : 2
    img_size=224
    color=3
    #이미지 데이터와 라벨(가위 : 0, 바위 : 1, 보 : 2) 데이터를 담을 행렬(matrix) 영역을 생성합니다.
    imgs=np.zeros(number_of_data*img_size*img_size*color,dtype=np.int32).reshape(number_of_data,img_size,img_size,color)
    labels=np.zeros(number_of_data,dtype=np.int32)

    idx=0
    for file in glob.iglob(img_path+'/scissors/*.jpg'):
        img = Image.open(file).resize((224, 224))  # 크기를 (28, 28)로 변경
        img = np.array(img, dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=0   # 가위 : 0
        idx=idx+1

    for file in glob.iglob(img_path+'/rock/*.jpg'):
        img = Image.open(file).resize((224, 224))  # 크기를 (28, 28)로 변경
        img = np.array(img, dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=1   # 바위 : 1
        idx=idx+1  
    
    for file in glob.iglob(img_path+'/paper/*.jpg'):
        img = Image.open(file).resize((224, 224))  # 크기를 (28, 28)로 변경
        img = np.array(img, dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=2   # 보 : 2
        idx=idx+1
        
    print("학습데이터(x_train)의 이미지 개수는", idx,"입니다.")
    return imgs, labels

image_dir_path = "dataset" #폴더명
(X, y)=load_data(image_dir_path)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=0, stratify=y)
x_train_norm = X_train/255.0   # 입력은 0~1 사이의 값으로 정규화
x_val_norm = X_val/255.0
print("x_train shape: {}".format(X_train.shape))
print("y_train shape: {}".format(y_train.shape))

학습데이터(x_train)의 이미지 개수는 7491 입니다.
x_train shape: (5243, 224, 224, 3)
y_train shape: (5243,)


In [56]:
# for i in range(100):
#     plt.imshow(x_train[i])
#     print('라벨 : ', y_train[i])
#     plt.show()

In [57]:
# model = keras.models.Sequential()
# model.add(keras.layers.Conv2D(64, (5,5), activation='relu', input_shape=(224,224,3)))  # 필터와 커널 크기 변경
# model.add(keras.layers.MaxPool2D(3,3))  # 풀링 크기 변경
# model.add(keras.layers.Conv2D(128, (3,3), activation='relu'))
# model.add(keras.layers.MaxPooling2D((2,2)))
# model.add(keras.layers.Flatten())
# model.add(keras.layers.Dense(256, activation='relu'))  # Dense 레이어 유닛 증가
# model.add(keras.layers.Dense(20, activation='softmax'))

In [58]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

model=keras.models.Sequential()
model.add(keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(224,224,3)))
model.add(keras.layers.MaxPool2D(2,2))
model.add(keras.layers.Conv2D(64, (3,3), activation='relu'))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.MaxPooling2D((2,2)))
# model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(128, activation='relu', kernel_regularizer=keras.regularizers.l2(0.01)))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(3, activation='softmax'))

model.summary()

c:\Users\zqrc0\anaconda3\envs\motionpy310\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_8 (Conv2D)               │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 109, 109, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 186624)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 128)            │    23,888,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,908,035 (91.20 MB)

 Trainable params: 23,907,907 (91.20 MB)

 Non-trainable params: 128 (512.00 B)

In [59]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator

optimizer = keras.optimizers.Adam(learning_rate=0.0005)

model.compile(optimizer=optimizer,
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

# ModelCheckpoint 콜백 설정
checkpoint = ModelCheckpoint(
    'dataset/model/best_model.keras',  # 경로를 지정하여 저장
    monitor='val_loss',
    save_best_only=True,
    mode='min'
)

# 데이터 증강
datagen = ImageDataGenerator(
    rotation_range=20,     # 이미지 회전
    width_shift_range=0.2, # 가로로 이동
    height_shift_range=0.2,# 세로로 이동
    shear_range=0.2,       # 시야각 변환
    zoom_range=0.2,        # 확대/축소
    horizontal_flip=True,  # 좌우 반전
    fill_mode='nearest'
)

# 증강된 이미지로 모델 학습
model.fit(datagen.flow(x_train_norm, y_train),
          validation_data=(x_val_norm, y_val),
          epochs=30,
          callbacks=[checkpoint])   # checkpoint 콜백 추가

c:\Users\zqrc0\anaconda3\envs\motionpy310\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30
164/164 ━━━━━━━━━━━━━━━━━━━━ 80s 465ms/step - accuracy: 0.4140 - loss: 5.9713 - val_accuracy: 0.3149 - val_loss: 2.2390
Epoch 2/30
164/164 ━━━━━━━━━━━━━━━━━━━━ 75s 457ms/step - accuracy: 0.5131 - loss: 1.9358 - val_accuracy: 0.4373 - val_loss: 1.6198
Epoch 3/30
164/164 ━━━━━━━━━━━━━━━━━━━━ 69s 417ms/step - accuracy: 0.5104 - loss: 1.5846 - val_accuracy: 0.6437 - val_loss: 1.3765
Epoch 4/30
164/164 ━━━━━━━━━━━━━━━━━━━━ 64s 392ms/step - accuracy: 0.5385 - loss: 1.4285 - val_accuracy: 0.3728 - val_loss: 1.9790
Epoch 5/30
164/164 ━━━━━━━━━━━━━━━━━━━━ 64s 393ms/step - accuracy: 0.5667 - loss: 1.3470 - val_accuracy: 0.6125 - val_loss: 1.2013
Epoch 6/30
164/164 ━━━━━━━━━━━━━━━━━━━━ 64s 390ms/step - accuracy: 0.5862 - loss: 1.2863 - val_accuracy: 0.5725 - val_loss: 1.2786
Epoch 7/30
164/164 ━━━━━━━━━━━━━━━━━━━━ 64s 390ms/step - accuracy: 0.5945 - loss: 1.2877 - val_accuracy: 0.5698 - val_loss: 1.4274
Epoch 8/30
164/164 ━━━━━━━━━━━━━━━━━━━━ 65s 398ms/step - accuracy: 0.6323 - loss: 1

In [60]:
image_dir_path = "dataset/test"
(x_test, y_test)=load_data(image_dir_path,328)
x_test_norm = x_test/255.0

print("x_test shape : {}".format(x_test.shape))
print("y_test shape : {}".format(y_test.shape))

학습데이터(x_train)의 이미지 개수는 328 입니다.
x_test shape : (328, 224, 224, 3)
y_test shape : (328,)


In [61]:
# 저장된 최적의 모델 로드
best_model = keras.models.load_model('dataset/model/best_model.keras')

# 테스트 데이터로 평가
test_loss, test_accuracy = best_model.evaluate(x_test_norm, y_test, verbose=2)
print("test_loss : {}".format(test_loss))
print("test_accuracy : {}".format(test_accuracy))

11/11 - 1s - 73ms/step - accuracy: 0.5549 - loss: 1.5674
test_loss : 1.5673636198043823
test_accuracy : 0.5548780560493469


In [62]:
# 테스트 데이터로 평가
test_loss, test_accuracy = best_model.evaluate(x_val_norm, y_val, verbose=2)
print("test_loss : {}".format(test_loss))
print("test_accuracy : {}".format(test_accuracy))

71/71 - 4s - 57ms/step - accuracy: 0.8630 - loss: 0.9710
test_loss : 0.9709633588790894
test_accuracy : 0.8629893064498901


In [37]:
predicted_result = model.predict(x_test_norm)	# model이 추론한 확률값
predicted_labels = np.argmax(predicted_result, axis=1)

idx=100		# 값을 변경해서 찾아보자
print('model.predict() 결과 : ', predicted_result[idx])
print('model이 추론한 가장 가능성이 높은 결과 : ', predicted_labels[idx])
print('실제 데이터의 라벨 : ', y_test[idx])

11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 51ms/step
model.predict() 결과 :  [0.7746478  0.13476472 0.09058755]
model이 추론한 가장 가능성이 높은 결과 :  0
실제 데이터의 라벨 :  0


In [ ]:
# plt.imshow(x_test[idx], cmap=plt.cm.binary)
# plt.show()

In [64]:
import random
wrong_predict_list=[]
for i, _ in enumerate(predicted_labels):
    if predicted_labels[i] != y_test[i]:
        wrong_predict_list.append(i)
        
samples = random.choices(population=wrong_predict_list, k=10) # k값을 수정하면 더 많은 값의 수를 볼 수 있다.

for n in samples:
    print("예측확률분포: " + str(predicted_result[n]))
    print("라벨: " + str(y_test[n]) + ", 예측결과: " + str(predicted_labels[n]))
    # plt.imshow(x_test[n], cmap=plt.cm.binary)
    print(n)
    # plt.show()

예측확률분포: [0.70132756 0.17618598 0.12248644]
라벨: 1, 예측결과: 0
168
예측확률분포: [0.627947   0.20508824 0.16696471]
라벨: 2, 예측결과: 0
319
예측확률분포: [0.39400688 0.3466109  0.25938222]
라벨: 1, 예측결과: 0
193
예측확률분포: [0.3276664  0.3454967  0.32683685]
라벨: 2, 예측결과: 1
311
예측확률분포: [0.78082305 0.11518728 0.10398969]
라벨: 2, 예측결과: 0
282
예측확률분포: [0.6769336  0.19874091 0.12432552]
라벨: 2, 예측결과: 0
284
예측확률분포: [0.6183745  0.22572798 0.15589751]
라벨: 1, 예측결과: 0
196
예측확률분포: [0.5059472  0.28466043 0.20939243]
라벨: 1, 예측결과: 0
161
예측확률분포: [0.43940514 0.3017701  0.25882468]
라벨: 2, 예측결과: 0
302
예측확률분포: [0.4781103  0.26698735 0.25490227]
라벨: 2, 예측결과: 0
280


In [65]:
model.save('dataset/model/model.keras', include_optimizer=False)